In [14]:
from IPython.display import display
import pandas as pd
import math
from collections import defaultdict

In [15]:
df=pd.read_csv("Data.csv")
res_col="Play"    # result column
res_val=set(df[res_col])    # to store all result values
df

,Outlook,Termperature Numeric,Temperature Nominal,Humidity Numeric,Humidity Nominal,Windy,Play
0,overcast,83,hot,86,high,False,yes
1,overcast,64,cool,65,normal,True,yes
2,overcast,72,mild,90,high,True,yes
3,overcast,81,hot,75,normal,False,yes
4,rainy,70,mild,96,high,False,yes
5,rainy,68,cool,80,normal,False,yes
6,rainy,65,cool,70,normal,True,no
7,rainy,75,mild,80,normal,False,yes
8,rainy,71,mild,91,high,True,no
9,sunny,85,hot,85,high,False,no


In [16]:
# drop columns which won't be considered for calculation

df.drop(["Termperature Numeric", "Humidity Numeric"],axis=1, inplace=True)
df

,Outlook,Temperature Nominal,Humidity Nominal,Windy,Play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


In [17]:
# function to count total number of each value

def cnt_values(lis):
    d=defaultdict(int)
    for val in lis:
        d[val]+=1
    return d

In [18]:
# function to calculate entropy

def entropy(lis):
    d=lis.value_counts()
    tot=len(lis)
    
    # if count of any value is 0, entropy becomes 0
    if (len(d)!=len(res_val)):
        return 0
    
    ans=0
        
    # calculate entropy
    for val in d:
        temp=(val/tot)*math.log2(val/tot)
        ans-=temp
    return round(ans,4)
        
    

In [19]:
# function to calculate gain of each column

def gain(data, col):
    ans=entropy(data[res_col])  # entropy of result column

    # calcuate cnt of each value
    d=cnt_values(data[col])
    
    # calculate entropy of each value in a column
    for k,v in d.items():
        ent=entropy(data[data[col]==k][res_col])
        temp=v*ent/len(data)
        ans-=temp
    
    return round(ans,4)
    

In [20]:
# function to calculate maximum gain among all columns

def max_gain(data):
    mx_g=float("-inf")
    g_col=None
    
    d={"Column":[], "Gain":[]}
    # traverse all columns in data frame
    for col in data.columns:
        if (col==res_col):
            continue
        
        # calculate gain of each column
        g=gain(data, col)
        d["Column"].append(col)
        d["Gain"].append(g)
        # if gain is maximum store it
        if (g>mx_g):
            mx_g=g
            g_col=col
    display(pd.DataFrame(d))
    return g_col
                

In [21]:
class Node:
    def __init__(self, val):
        self.val=val
        self.child={}

In [22]:
# function to create tree

def tree(data):
    
    # find col with max gain
    g=max_gain(data)

    # create a node of that col
    node=Node(g)
    
    # get all unique values in that column
    d=cnt_values(data[g])
    
    # if there are no more columns left
    if (len(data.columns)==2):
        
        # check for each value
        for k in d:
            mx, att=0, None
            t=data[data[g]==k][res_col]
            
            # check for maximum occurrence
            for val in res_val:
                cnt=t.value_counts()[val]
                if (cnt>=mx):
                    mx=cnt
                    att=val
            
            new_node=Node(att)
            node.child[k]=new_node
        return node

    for k in d:
        temp=data[data[g]==k]

        # check if there is only single value in result
        if (len(temp[res_col].value_counts())==1):
            new_node=Node(temp[res_col].iloc[0])
            node.child[k]=new_node
            
        # else recusively call function on remaining columns
        else:
            print("Attribute :",k, "/ Parent :",g)
            node.child[k]=tree(temp.drop(g, axis=1))
    return node
    

In [23]:
root=tree(df)

,Column,Gain
0,Outlook,0.2467
1,Temperature Nominal,0.0292
2,Humidity Nominal,0.1519
3,Windy,0.0481


Attribute : rainy / Parent : Outlook


,Column,Gain
0,Temperature Nominal,0.020
1,Humidity Nominal,0.020
2,Windy,0.971


Attribute : sunny / Parent : Outlook


,Column,Gain
0,Temperature Nominal,0.571
1,Humidity Nominal,0.971
2,Windy,0.020


In [24]:
print("Root :",root.val)

Root : Outlook


In [25]:
def level_order(node):
    q=[node]
    while (q):
        sz=len(q)
        for _ in range (sz):
            t=q.pop(0)
            print("parent :",t.val)
            for k,v in t.child.items():
                print ("{}->{}".format(k,v.val), end="\t")
                q.append(v)
                
            print("\n")            
            

In [26]:
level_order(root)

parent : Outlook
overcast->yes	rainy->Windy	sunny->Humidity Nominal	

parent : yes


parent : Windy
False->yes	True->no	

parent : Humidity Nominal
high->no	normal->yes	

parent : yes


parent : no


parent : no


parent : yes


